<a href="https://colab.research.google.com/github/igedemadepradnyaairlangga/Dataanalitycs/blob/master/klasifikasi_gambar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Menghubungkan dataset ke drive

In [42]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [43]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [44]:
#changing the working directory
%cd /content/gdrive/My Drive/Kaggle
#Check the present working directory using pwd command

/content/gdrive/My Drive/Kaggle


In [8]:
!ls

 amazon_prime_titles.csv	 kaggle.json
 combined_dataset.csv		 Mall_Customers.csv
 day.csv			 rps-cv-images
'e-shop clothing 2008.csv'	'Salinan amazon_prime_titles.csv'
'e-shop clothing 2008.csv.zip'	 udemy_output_All_Finance__Accounting_p1_p626.csv
 hour.csv


libary

In [57]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
import cv2
from google.colab import files
from ipywidgets import FileUpload
import io
from PIL import Image

path direktori ke dataset

In [48]:
# Path ke direktori dataset
dataset_dir = '/content/gdrive/My Drive/Kaggle/rps-cv-images'
train_data_dir = os.path.join(dataset_dir, 'train')
validation_data_dir = os.path.join(dataset_dir, 'validation')

parameter dan inisiasi model

In [62]:
# Parameter-model
input_shape = (150, 150, 3)
num_classes = 3

# Inisialisasi model sequential
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Augmentasi data gambar
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0/255)

# Parameter training
batch_size = 32

# Memuat dataset Anda ke dalam variabel X dan y
X = []
y = []

# Isi X dan y dengan data gambar dan label Anda

# Daftar kategori (misalnya: "paper", "rock", "scissors")
categories = ['scissors', 'rock', 'paper']

# Memuat data gambar dan label dari direktori dataset
for category in categories:
    category_path = os.path.join(dataset_dir, category)
    class_label = categories.index(category)  # Label kelas (misalnya, 0, 1, 2)

    for img_file in os.listdir(category_path):
        if img_file.endswith(".jpg") or img_file.endswith(".jpeg") or img_file.endswith(".png"):
            img_path = os.path.join(category_path, img_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (150, 150))  # Sesuaikan dengan ukuran yang Anda inginkan
            X.append(img)
            y.append(class_label)

# Konversi ke array NumPy
X = np.array(X)
y = np.array(y)

# Mengubah label ke one-hot encoding
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=3)

# Bagi dataset menjadi train set dan validation set (40% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
validation_generator = test_datagen.flow(X_val, y_val, batch_size=batch_size)

# Training model (maksimal 30 menit)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

epochs = 100  # Anda dapat menyesuaikan jumlah epoch sesuai kebutuhan
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(X_val) // batch_size,
    callbacks=[early_stopping, model_checkpoint])

# Memuat model terbaik
model.load_weights('best_model.h5')

# Widget "Choose File" untuk memilih file gambar
uploaded = files.upload()

if uploaded:
    # Pilih file gambar yang diunggah
    uploaded_image_path = list(uploaded.keys())[0]

    # Memproses gambar yang diunggah
    uploaded_image = Image.open(io.BytesIO(uploaded[uploaded_image_path]))
    uploaded_image = uploaded_image.resize((150, 150))
    uploaded_image = np.array(uploaded_image)
    uploaded_image = np.expand_dims(uploaded_image, axis=0)

    # Melakukan prediksi pada gambar yang diunggah
    prediction = model.predict(uploaded_image)
    labels = ['scissors', 'rock', 'paper']
    predicted_label = labels[np.argmax(prediction)]

    print("Hasil prediksi:", predicted_label)
else:
    print("Belum ada file yang diunggah.")


Epoch 1/100
41/41 [==============================] - 56s 1s/step - loss: 1.0585 - accuracy: 0.4345 - val_loss: 0.8243 - val_accuracy: 0.7500
Epoch 2/100
41/41 [==============================] - 56s 1s/step - loss: 0.7417 - accuracy: 0.6921 - val_loss: 0.5211 - val_accuracy: 0.8310
Epoch 3/100
41/41 [==============================] - 57s 1s/step - loss: 0.5122 - accuracy: 0.7950 - val_loss: 0.3760 - val_accuracy: 0.8380
Epoch 4/100
41/41 [==============================] - 58s 1s/step - loss: 0.3990 - accuracy: 0.8529 - val_loss: 0.2427 - val_accuracy: 0.9248
Epoch 5/100
41/41 [==============================] - 58s 1s/step - loss: 0.3768 - accuracy: 0.8514 - val_loss: 0.2215 - val_accuracy: 0.9387
Epoch 6/100
41/41 [==============================] - 55s 1s/step - loss: 0.3006 - accuracy: 0.8765 - val_loss: 0.1326 - val_accuracy: 0.9595
Epoch 7/100
41/41 [==============================] - 56s 1s/step - loss: 0.2544 - accuracy: 0.9146 - val_loss: 0.1865 - val_accuracy: 0.9271
Epoch 8/100
4

Saving 0a3UtNzl5Ll3sq8K.png to 0a3UtNzl5Ll3sq8K (1).png
1/1 [==============================] - 0s 101ms/step
Hasil prediksi: paper


In [52]:
print(len(X))
print(len(y))


2188
2188
